In [1]:
from time import time
import numpy as np
import numba as nb

In [2]:
@nb.njit('f4[::1](f4[::1])')
def eager_test(x):
    return (np.cos(x) + 2) / np.sin(x)

In [3]:
@nb.njit
def lazy_test(x):
    return (np.cos(x) + 2) / np.sin(x)

In [4]:
y = np.random.normal(10, 2, (1000000,))
y2 = y.astype(np.float32)

In [5]:
start_time = time()
lazy_test(y2)
print(time() - start_time)

0.08684635162353516


In [6]:
start_time = time()
lazy_test(y2)
print(time() - start_time)

0.005002021789550781


In [7]:
start_time = time()
eager_test(y2)
print(time() - start_time)

0.005011796951293945


In [8]:
start_time = time()
eager_test(y2)
print(time() - start_time)

0.004999637603759766


In [9]:
lazy_test.signatures

[(array(float32, 1d, C),)]

In [10]:
eager_test.signatures

[(array(float32, 1d, C),)]

In [11]:
@nb.experimental.jitclass({'a': nb.int32})
class TestClass(object):
    
    def __init__(self, a):
        self.a = 2.0
        
    def get(self):
        return self.a * 5

In [103]:
TestClass.class_type.instance_type

instance.jitclass.TestClass#2258ffb6850<a:int32>

In [107]:
@nb.njit(nb.int64(TestClass.class_type.instance_type))
def double_class(a):
    return a.get() * 3

In [108]:
double_class(TestClass(2))

30

In [106]:
double_class.nopython_signatures

[(instance.jitclass.TestClass#2258ffb6850<a:int32>,) -> int64]

In [19]:
double_class.signatures

[(instance.jitclass.TestClass#2258fcf16a0<a:int32>,)]

In [129]:
nonparametric_sig = ['f8[::1](f8[:, ::1], i4[::1], i4, f8)', 'f4[::1](f4[:, ::1], i4[::1], i4, f4)']
@nb.njit(nonparametric_sig, fastmath=True)
def nonparametric_cost(x, start, end, c):
    _d = (end - start).astype(x.dtype)
    f = ((x[end, :] - x[start, :]).T * x.dtype.type(0.5)) / _d
    _t = _d * (f * np.log(f) + (1 - f) * np.log(1 - f)).sum(axis=0)
    return c * _t

@nb.njit(['f8[:, ::1](f8[::1], i4)', 'f4[:, ::1](f4[::1], i4)'], fastmath=True)
def create_partial_sums(x, k):

    n = x.shape[0]
    partial_sums = np.zeros(shape=(n + 1, k), dtype=x.dtype)
    sorted_data = np.sort(x)

    for i in np.arange(k):

        z = -1 + (2 * i + 1.0) / k
        p = 1.0 / (1 + (2 * n - 1) ** (-z))
        t = sorted_data[int((n - 1) * p)]

        for j in np.arange(1, n + 1):

            partial_sums[j, i] = partial_sums[j - 1, i]
            if x[j - 1] < t:
                partial_sums[j, i] += 2
            if x[j - 1] == t:
                partial_sums[j, i] += 1
    return partial_sums

In [147]:
nb.typeof(create_partial_sums)

type(CPUDispatcher(<function create_partial_sums at 0x00000225903299D0>))

In [166]:
@nb.experimental.jitclass({'k': nb.int32,
                           'c': nb.float32,
                           'stats': nb.float32[:, :],
                           'preprocess_fn': nb.typeof(create_partial_sums),
                           'cost_fn': nb.typeof(nonparametric_cost)})
class JitNonParametricCost:

    def __init__(self, k):
        self.k = k
        self.c = 0.0
        self.preprocess_fn = create_partial_sums
        self.cost_fn = nonparametric_cost

    def fit(self, signal):
        self.stats = self.preprocess_fn(signal, self.k)
        self.c = np.float32(2.0 * (-np.log(2 * signal.shape[0] - 1) / self.k))

    def error(self, start, end):
        return self.cost_fn(self.stats, start, end, self.c)

In [239]:
nonparametric_sig = ['f8[::1](f8[:, ::1], i4[::1], i4)', 'f4[::1](f4[:, ::1], i4[::1], i4)']
@nb.njit(nonparametric_sig, fastmath=True, nogil=True)
def nonparametric_cost(x, start, end):
    _d = (end - start).astype(x.dtype)
    f = ((x[end, :] - x[start, :]).T * x.dtype.type(0.5)) / _d
    _t = _d * (f * np.log(f) + (1 - f) * np.log(1 - f)).sum(axis=0)
    return _t

c = 2.0

@nb.njit(nonparametric_sig, fastmath=True, nogil=True)
def factory_fn(x, start, stop):
    return nonparametric_cost(x, start, stop) * np.float32(c)

In [ ]:
types.float64(types.float64, types.float64)

In [407]:
nb.float32[::1](nb.types.FunctionType(nb.float32[::1](nb.float32[:, ::1], nb.int32[::1], nb.int32)))

(FunctionType[array(float32, 1d, C)(array(float32, 2d, C), array(int32, 1d, C), int32)],) -> array(float32, 1d, C)

In [408]:
@nb.njit([nb.float32[::1](nb.types.FunctionType(nb.float32[::1](nb.float32[:, ::1], nb.int32[::1], nb.int32)))])
def fn(a):
    return a(np.float32([[1.0, 1.0], [2.0, 2.0]]), np.int32([10]), np.int32(200))

In [2]:
from numba.typed import List

nonparametric_sig = ['f8[::1](f8[:, ::1], i4[::1], i4)', 'f4[::1](f4[:, ::1], i4[::1], i4)']
@nb.njit(nonparametric_sig, fastmath=True)
def nonparametric_cost(x, start, end):
    _d = (end - start).astype(x.dtype)
    _seg = (x[end, :] - x[start, :]).T
    _is_valid = (_seg != 0) & (_seg != (_d * 2))
    f = (_seg * x.dtype.type(0.5)) / _d
    _h = _d * (f * np.log(f) + (1 - f) * np.log(1 - f))
    _t = np.where((_seg != 0) & (_seg != (_d * 2)), _h, x.dtype.type(0.0))
    return _t.sum(axis=0)

@nb.njit(['f8[:, ::1](f8[::1], i4)', 'f4[:, ::1](f4[::1], i4)'], fastmath=True)
def create_partial_sums(x, k):

    n = x.shape[0]
    partial_sums = np.zeros(shape=(n + 1, k), dtype=x.dtype)
    sorted_data = np.sort(x)

    for i in np.arange(k):

        z = -1 + (2 * i + 1.0) / k
        p = 1.0 / (1 + (2 * n - 1) ** (-z))
        t = sorted_data[int((n - 1) * p)]

        for j in np.arange(1, n + 1):

            partial_sums[j, i] = partial_sums[j - 1, i]
            if x[j - 1] < t:
                partial_sums[j, i] += 2
            if x[j - 1] == t:
                partial_sums[j, i] += 1
    return partial_sums


x = np.hstack([np.random.normal(2, 2, (1000,)), np.random.normal(-10, 1, (1250,)), np.random.normal(10, 2, (1500,))])
k = 20
c = 2.0 * (-np.log(2 * x.shape[0] - 1) / k)

@nb.njit(nonparametric_sig, fastmath=True)
def wrapped_fn(x, start, stop):
    return nonparametric_cost(x, start, stop) * x.dtype.type(c)

@nb.njit(['f8[:, ::1](f8[::1])', 'f4[:, ::1](f4[::1])'], fastmath=True)
def wrapped_preprocess_fn(x):
    return create_partial_sums(x, k)

In [35]:
# @nb.njit([nb.int32[::1](nb.float32[::1],
#                         nb.int32,
#                         nb.float32,
#                         nb.types.FunctionType(nb.float32[::1](nb.float32[:, ::1], nb.int32[::1], nb.int32)),
#                         nb.types.FunctionType(nb.float32[:, ::1](nb.float32[::1])))
#          ])
@nb.njit(fastmath=True, nogil=True)
def pelt(x, min_len, penalty, cost_fn, preprocess_fn):
    """Pruned exact linear time changepoint segmentation"""
    
    # Setting up summary statistics and objects
    n = np.int32(x.shape[0])
    #model.fit(x)
    sum_stats = preprocess_fn(x)
    
    # Initializing pelt parameters
    f = np.empty(shape=(n,), dtype=np.float32)
    f[0] = -penalty
    costs = np.empty(shape=(n,), dtype=np.float32)
    cp = List([np.int32([0]),])
    r = np.zeros(shape=(n,), dtype=np.bool_)
    r[0] = True

    # Entering main loop
    for tau_star in np.arange(1, n, dtype=np.int32):
        
        # Calculating minimum segment cost
        _r = np.flatnonzero(r).astype(np.int32)
        costs[_r] = cost_fn(sum_stats, _r, tau_star)

        #costs[r] = model.error(r, tau_star)
        _costs = costs[_r] + f[_r]
        
        f[tau_star] = _costs.min()
        tau_l = _r[np.argmin(_costs)]
        
        # Setting new changepoints
        x = np.hstack((cp[tau_l], np.int32([tau_l])))
        #x = cp[tau_l] + [tau_l]
        cp.append(x)
        
        # Setting new candidate points
        #r = np.append(r[(f[r]) <= (f[tau_star] + penalty)], tau_star)
        r[:] = False
        r[_r[(f[_r]) <= (f[tau_star] + penalty)]] = True
        r[tau_star] = True
        
    return cp[-1][2:]

In [4]:
%timeit pelt.py_func(x, 10, 100.0, wrapped_fn, wrapped_preprocess_fn)

7.27 s ± 260 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
pelt(x.astype(np.float32), 10, 100.0, wrapped_fn, wrapped_preprocess_fn)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of type(CPUDispatcher(<function wrapped_fn at 0x00000226CF8AFAF0>)) with parameters (array(float32, 2d, C), array(int64, 1d, C), int32)
Known signatures:
 * (array(float64, 2d, C), array(int32, 1d, C), int32) -> array(float64, 1d, C)
 * (array(float32, 2d, C), array(int32, 1d, C), int32) -> array(float32, 1d, C)[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function wrapped_fn at 0x00000226CF8AFAF0>))[0m
[0m[1mDuring: typing of call at <ipython-input-27-8195c5c7d8d3> (29)
[0m
[1m
File "<ipython-input-27-8195c5c7d8d3>", line 29:[0m
[1mdef pelt(x, min_len, penalty, cost_fn, preprocess_fn):
    <source elided>
        _r = np.flatnonzero(r)
[1m        costs[_r] = cost_fn(sum_stats, _r, tau_star)
[0m        [1m^[0m[0m


In [39]:
%timeit pelt(x.astype(np.float32), 10, 100.0, wrapped_fn, wrapped_preprocess_fn)

6.63 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects

# import R's utility package
utils = rpackages.importr('utils')
rcp = rpackages.importr('changepoint')
rcpnp = rpackages.importr('changepoint.np')

r_series = robjects.FloatVector(np.array(x))

In [38]:
%timeit rcpnp.cpt_np(r_series, penalty="Manual", pen_value=100.0, method='PELT', nquantiles=k)

3.18 s ± 75.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
py_pelt = pelt.py_func
%load_ext line_profiler

In [29]:
def wrapped_fn(x, start, stop):
    return nonparametric_cost.py_func(x, start, stop) * c

In [41]:
%lprun -f nonparametric_cost.py_func py_pelt(x, 10, 100.0, nonparametric_cost.py_func, wrapped_preprocess_fn)

<ipython-input-2-783dfd7e7493>:10: RuntimeWarning: divide by zero encountered in log
  _h = _d * (f * np.log(f) + (1 - f) * np.log(1 - f))
<ipython-input-2-783dfd7e7493>:10: RuntimeWarning: invalid value encountered in multiply
  _h = _d * (f * np.log(f) + (1 - f) * np.log(1 - f))


In [44]:
parametric_sig = ['f8[::1](f8[:, ::1], i4[::1], i4)', 'f4[::1](f4[:, ::1], i4[::1], i4)']
#@nb.njit(parametric_sig, fastmath=True)
@nb.njit(fastmath=True)
def normal_mean_var_cost(x, start, end):
    _x = x[end, :] - x[start, :]
    _pi_const = x.dtype.type(np.log(2 * np.pi))
    return _x[:, 3] * (_pi_const + np.log(np.fmax((_x[:, 1] - ((_x[:, 0] * _x[:, 0]) / _x[:, 3]))/ _x[:, 3], x.dtype.type(1e-8)) + 1))

#@nb.njit(parametric_sig, fastmath=True)
@nb.njit(fastmath=True)
def wrapped_param_fn(x, start, stop):
    return normal_mean_var_cost(x, start, stop) + x.dtype.type(100.0)

#@nb.njit(['f8[:, ::1](f8[::1])', 'f4[:, ::1](f4[::1])'], fastmath=True)
@nb.njit(fastmath=True)
def create_summary_stats(x):
    n = x.shape[0]
    start_val = x.dtype.type(0)
    sum_stats = np.stack((np.append(start_val, x.cumsum()),
                          np.append(start_val, (x ** 2).cumsum()),
                          np.append(start_val, ((x - x.mean()) ** 2).cumsum()),
                          np.arange(0, n + 1, dtype=x.dtype)
                         ),
                         axis=-1)
    return sum_stats

In [45]:
pelt(x.astype(np.float32), 10, 100.0, wrapped_param_fn, create_summary_stats)

array([1000, 2250])

In [46]:
%timeit pelt(x.astype(np.float32), 10, 100.0, wrapped_param_fn, create_summary_stats)

224 ms ± 3.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%lprun -f pelt.py_func pelt.py_func(x.astype(np.float32), 10, 100.0, normal_mean_var_cost, create_summary_stats)

In [13]:
%timeit np.array(rcp.cpt_meanvar(r_series, penalty="Manual", pen_value=100.0, method='PELT', test_stat='Normal'))

105 ms ± 948 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
